# Using Awkward Arrays with `cuda.compute`

This notebook shows how to use the facilities provided by the [`cuda.compute`](https://nvidia.github.io/cccl/python/compute.html) library to accelerate operations on Awkward Arrays on GPUs.

Note that Awkward Arrays [already support](https://awkward-array.org/doc/main/user-guide/how-to-math-gpu.html) execution of operations on the GPU via its CUDA backend, which is already quite a bit faster than CPU execution. We'll show how you can use `cuda.compute` to get **even more** out of your GPU.

Our hope is that eventually, parts of Awkward Array can use `cuda.compute` to give you the same benefits without having to interact with another library. 

## Example 1: Reductions with Awkward Arrays

This example builds upon the Awkward Array documentation showing [how to use Numba CUDA kernels](https://awkward-array.org/doc/main/user-guide/how-to-use-in-numba-cuda.html#how-to-use-awkward-arrays-in-numba-s-cuda-target) to operate on a ragged array.


In [1]:
import awkward as ak
import cupy as cp
import numpy as np
import numba
import numba.cuda

from ak_helpers import *

First, let's build an array with some random data:

In [2]:
N = 2**20

counts = ak.Array(cp.random.randint(0, 256, size=N, dtype=np.int32))
content = ak.Array(cp.random.normal(0, 45.0, int(ak.sum(counts))).astype(np.float32))
array = ak.unflatten(content, counts)
array

<Array [[], ..., [-71.310905, ..., -7.7317753]] type='1048576 * var * float32'>

### Using a Numba CUDA kernel

As described in the docs, we can write a custom CUDA kernel using Numba to compute list sums. Awkward arrays can be registered with Numba and used within CUDA kernels:

In [3]:
ak.numba.register_and_check()

In [4]:
@numba.cuda.jit(extensions=[ak.numba.cuda])
def path_length(out, array):
    tid = numba.cuda.grid(1)
    if tid < len(array):
        out[tid] = 0
        for i, x in enumerate(array[tid]):
            out[tid] += x

In [5]:
blocksize = 256
numblocks = (N + blocksize - 1) // blocksize

result = cp.empty(len(array), dtype=np.float32)
path_length[numblocks, blocksize](result, array)
print(result)

[   0.        -203.49106   -421.83444   ...    5.5652122  454.74536
 -165.9854   ]


This is already quite a bit faster than using the CPU:

In [6]:
%%timeit
path_length[numblocks, blocksize](result, array)
cp.cuda.Device().synchronize()

2.41 ms ± 11.8 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [7]:
cpu_array = ak.to_backend(array, "cpu")

In [8]:
%%timeit
ak.sum(cpu_array)

18.4 ms ± 74.7 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)


### Using `cuda.compute`

We can do the same thing using the [segmented_reduce](https://nvidia.github.io/cccl/python/compute_api.html#cuda.compute.algorithms.segmented_reduce) function from the `cuda.compute` library.

In [9]:
from cuda.compute import segmented_reduce, OpKind

First, we have to grab the CuPy arrays underlying the awkward array. We use a helper for this:

In [10]:
offsets, data = get_example1_buffers(array)

Next, we prepare the remaining arguments to `segmented_reduce`:

In [11]:
d_out = cp.empty_like(result)
h_init = np.zeros(shape=(1,), dtype=result.dtype)
num_segments = len(array)

Now we can call `segmented_reduce`, which computes the list sums:

In [12]:
segmented_reduce(
    d_in=data,
    d_out=d_out,
    start_offsets_in=offsets[:-1],
    end_offsets_in=offsets[1:],
    op=OpKind.PLUS,
    h_init=h_init,
    num_segments=num_segments)

We can see that the results are identical to those produced using our Numba CUDA kernel from before:

In [13]:
print(d_out)

[   0.       -203.49103  -421.83447  ...    5.565201  454.74527
 -165.98532 ]


No CUDA kernel necessary! And we get better performance:

In [14]:
%%timeit -n 10 -r 10

segmented_reduce(
    d_in=data,
    d_out=d_out,
    start_offsets_in=offsets[:-1],
    end_offsets_in=offsets[1:],
    op=OpKind.PLUS,
    h_init=h_init,
    num_segments=num_segments)
cp.cuda.Device().synchronize()

1.23 ms ± 8.01 μs per loop (mean ± std. dev. of 10 runs, 10 loops each)


## Example 2

Next, we'll use the [dimuon search example](https://awkward-array.org/doc/main/getting-started/jagged-ragged-awkward-arrays.html#application-to-dimuons) from the Awkward Array documentation, to see how to accelerate something a bit more practical.

Note that in the code below, we blow up the input data by a factor of 100 (using `ak.concatenate`) to simulate a larger dataset:

In [15]:
import awkward as ak
import numpy as np
import cupy as cp
import uproot

file = uproot.open(
    "https://github.com/jpivarski-talks/2023-12-18-hsf-india-tutorial-bhubaneswar/raw/main/data/SMHiggsToZZTo4L.root"
)
tree = file["Events"]
arrays = tree.arrays(filter_name="/Muon_(pt|eta|phi|charge)/")
muons = ak.zip(
    {
        "pt": arrays["Muon_pt"],
        "eta": arrays["Muon_eta"],
        "phi": arrays["Muon_phi"],
        "charge": arrays["Muon_charge"],
    }
)
muons = ak.concatenate([muons] * 100)

pairs = ak.combinations(muons, 2)
pairs

<Array [[({...}, {...}), ..., (..., ...)], ...] type='29997300 * var * ({pt...'>

### Mass calculation using array operations (CPU)

First, we'll compute the invariant masses using array operations supported by awkward.  

In [16]:
mu1, mu2 = ak.unzip(pairs)
mass = np.sqrt(
    2 * mu1.pt * mu2.pt * (np.cosh(mu1.eta - mu2.eta) - np.cos(mu1.phi - mu2.phi))
)
print(mass)

[[89.5, 27, 22.5], [], [], ..., [2.39, 0.254, 0.505, ..., 2.93, 0.709], [], []]


Let's time the mass calculation:

In [17]:
%%timeit
mass = np.sqrt(
    2 * mu1.pt * mu2.pt * (np.cosh(mu1.eta - mu2.eta) - np.cos(mu1.phi - mu2.phi))
)

633 ms ± 1.75 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


### Mass calculation using array operations (GPU)

Awkward's CUDA support makes it very easy to perform the same computation using the GPU:

In [18]:
pairs = ak.to_backend(pairs, "cuda")
mu1, mu2 = ak.unzip(pairs)
mass = np.sqrt(
    2 * mu1.pt * mu2.pt * (np.cosh(mu1.eta - mu2.eta) - np.cos(mu1.phi - mu2.phi))
)
print(mass)

[[89.49166, 27.030632, 22.497808], [], [], [...], ..., [2.386485, ...], [], []]


We see that this is now much faster:

In [19]:
%%timeit
mass = np.sqrt(
    2 * mu1.pt * mu2.pt * (np.cosh(mu1.eta - mu2.eta) - np.cos(mu1.phi - mu2.phi))
)

43 ms ± 282 μs per loop (mean ± std. dev. of 7 runs, 10 loops each)


### Mass calculation using `cuda.compute`

In the code below, we use `cuda.compute` to do the same computation. We'll use the [`binary_transform`](https://nvidia.github.io/cccl/python/compute_api.html#cuda.compute.algorithms.binary_transform) operation to perform the computation.

In [20]:
from cuda.compute import PermutationIterator, ZipIterator, gpu_struct, binary_transform
from ak_helpers import *

The awkward arrays `mu1` and `mu2` are **indexed** arrays of **records**. In `cuda.compute` those concepts correspond to [`PermutationIterator`](https://nvidia.github.io/cccl/python/compute.html#iterators) and [`ZipIterator`](https://nvidia.github.io/cccl/python/compute_api.html#cuda.compute.iterators.ZipIterator) respectively. We extract the buffers out of `mu1` and `mu2`, and construct iterators out of them. These will be the inputs to `binary_transform`:

In [21]:
# extract the buffers (zero-copy) from the awkward arrays:
offsets1, index1, pt1, eta1, phi1, charge1 = get_example2_buffers(mu1)
offsets2, index2, pt2, eta2, phi2, charge2 = get_example2_buffers(mu2)

# construct the inputs to `binary_transform`:
d_in1 = PermutationIterator(ZipIterator(pt1, eta1, phi1, charge1), index1)
d_in2 = PermutationIterator(ZipIterator(pt2, eta2, phi2, charge2), index2)

Next, we'll create an "empty" awkward array that will hold the result (`mass`). We cheat a bit here and use a helper, as there is no `ak.empty_like` function because Awkward Arrays are "immutable":

In [22]:
# create an Awkward array to hold the result:
mass = make_like_offsets(mu1.pt)

# grab the buffer underneath that array:
d_out = mass.layout.content.data

Now, we're ready to call `binary_transform`. Our binary operator takes two "Muons" as arguments. We need to define a type corresponding to "Muon", and the operation that computes the invariant mass given two "Muons":

In [23]:
# A gpu_struct describing the data type of each element of our zip_iterator
Muon = gpu_struct({
        "pt": np.float32,
        "eta": np.float32,
        "phi": np.float32,
        "charge": np.int32
    })

# Define the binary operation that computes the mass:
def op(mu1: Muon, mu2: Muon) -> np.float32:
    return (
        2 * mu1.pt * mu2.pt * (np.cosh(mu1.eta - mu2.eta) - np.cos(mu1.phi - mu2.phi))
    )**0.5          

# Actually perform the binary_transform:
binary_transform(d_in1, d_in2, d_out, op, len(index1))
cp.cuda.Device().synchronize()

print(mass)

[[89.49166, 27.030634, 22.497808], [], [], [...], ..., [2.386485, ...], [], []]


We see that the result is the same as we got using Awkward's array operations.

Let's use `%%timeit` to see how all this compares to our previous approach that used the GPU: 

In [24]:
%%timeit
offsets1, index1, pt1, eta1, phi1, charge1 = get_example2_buffers(mu1)
offsets2, index2, pt2, eta2, phi2, charge2 = get_example2_buffers(mu2)

d_in1 = PermutationIterator(ZipIterator(pt1, eta1, phi1, charge1), index1)
d_in2 = PermutationIterator(ZipIterator(pt2, eta2, phi2, charge2), index2)

mass = make_like_offsets(mu1.pt)

d_out = mass.layout.content.data

Muon = gpu_struct({
        "pt": np.float32,
        "eta": np.float32,
        "phi": np.float32,
        "charge": np.int32
    })

def op(mu1: Muon, mu2: Muon) -> np.float32:
    return (
        2 * mu1.pt * mu2.pt * (np.cosh(mu1.eta - mu2.eta) - np.cos(mu1.phi - mu2.phi))
    )**0.5          

binary_transform(d_in1, d_in2, d_out, op, len(index1))
cp.cuda.Device().synchronize()

11.2 ms ± 1.13 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)


We see that using `cuda.compute` to do this computation is much faster.

## Why is it faster?

Recall that using the Awkward Array API, the mass computation was just:

```python
mass = np.sqrt(
    2 * mu1.pt * mu2.pt * (np.cosh(mu1.eta - mu2.eta) - np.cos(mu1.phi - mu2.phi))
)
```

This line of code is deceptively simple! There's actually a lot going on underneath. For example:

* Each arithmetic operation (e.g., multiplication, subtraction) launches a separate CUDA kernel
* Expressions like `mu1.pt` involve one or more CUDA kernels to materialize [indexed](https://awkward-array.org/doc/main/reference/generated/ak.contents.IndexedArray.html#ak-contents-indexedarray) arrays
* Each operation results in the creation of a new array to hold the result.

In contrast, `cuda.compute` fuses together all the above operations into a single kernel (invoked when you call `binary_transform`). Indexed arrays are never materialized, and no intermediate memory allocations need to be made. Note that we used _iterators_ as the inputs to `binary_transform` rather than arrays - **this is key**, and what enables fusing together of all the operations.

### Profiling

The command below uses [Nsight Systems](https://docs.nvidia.com/nsight-systems/UserGuide/index.html) to collect a performance profile of the script `mass_awkward.py`. This script contains the same code as before for computing invariant masses using the Awkward Array API. We're only interested in the section of code that does the mass calculation, so we've annotated that section of code in the script.

In [25]:
!nsys profile \
  -t cuda,nvtx \
  --capture-range=nvtx \
  --nvtx-capture="mass_calculation" \
  --env-var=NSYS_NVTX_PROFILER_REGISTER_ONLY=0 \
  -o mass_awkward \
  --force-overwrite true \
  --export sqlite \
  python mass_awkward.py

         This may increase runtime overhead and the likelihood of false
         dependencies across CUDA Streams. If you wish to avoid this, please
         disable the feature with --cuda-event-trace=false.
Try the 'nsys status --environment' command to learn more.

Try the 'nsys status --environment' command to learn more.

Processing events...
Generated:
	No reports were generated


--

We can visualize the results of profiling using [nsightful](https://github.com/brycelelbach/nsightful):

In [26]:
import nsightful

nsightful.display_nsys_sqlite_file_in_notebook('mass_awkward.sqlite')

Zoom in to the `mass_calculations` section of the profile, which is the interesting part. You'll notice that this section of code is composed of many smaller CUDA kernels:

<img src="images/mass-awkward-timeline.png" width="100%">

<img src="images/mass-awkward-kernels.png" width="100%">

--

Now, we repeat the same with the script `mass_cuda_compute.py`:

In [27]:
!nsys profile \
  -t cuda,nvtx \
  --capture-range=nvtx \
  --nvtx-capture="mass_calculation" \
  --env-var=NSYS_NVTX_PROFILER_REGISTER_ONLY=0 \
  -o mass_cuda_compute \
  --force-overwrite true \
  --export sqlite \
  python mass_cuda_compute.py

         This may increase runtime overhead and the likelihood of false
         dependencies across CUDA Streams. If you wish to avoid this, please
         disable the feature with --cuda-event-trace=false.
Try the 'nsys status --environment' command to learn more.

Try the 'nsys status --environment' command to learn more.

Capture range started in the application.
Capture range ended in the application.
Generating '/tmp/nsys-report-a911.qdstrm'
[1/2] [0%                          ] mass_cuda_compute.nsys-repProcessing events...
[1/2] [========================100%] mass_cuda_compute.nsys-rep
[2/2] [========================100%] mass_cuda_compute.sqlite
Generated:
	/home/ashwin/workspace/awkward-with-cccl/mass_cuda_compute.nsys-rep
	/home/ashwin/workspace/awkward-with-cccl/mass_cuda_compute.sqlite


In [28]:
import nsightful

nsightful.display_nsys_sqlite_file_in_notebook('mass_cuda_compute.sqlite')

This time, we see that everything has been fused into a single `transform_kernel`:

<img src="images/mass-cuda-compute-timeline.png" width="100%">

<img src="images/mass-cuda-compute-kernels.png" width="100%">